In [ ]:
#Cargar los datos
#Librerias 
#Cargar las librerias para procesar los datos
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn
%matplotlib inline
import time
from tensorflow import keras
from tensorflow.keras import layers
import scipy
import statsmodels
import sklearn
import seaborn as sns

In [ ]:
#Ruta de la carpeta base
wdirectory = os.getcwd()
wdirectory = wdirectory + '\low_freq'
print(f'{wdirectory}')
list_dir = os.listdir(wdirectory)
print(list_dir)

In [ ]:
#Casas en low_freq
casas = [casa for casa in list_dir if casa.find('house_', 0, len(casa)) != -1]
print(casas)

In [ ]:
#Guardar direcciones de cada casa
dir_casas = [os.path.join(wdirectory, casa) for casa in casas]
print(dir_casas)

In [ ]:
#Guardar ruta de direcciones de los dispositivos chanel de cada casa
def directorio_dispositivos(casa):
    #Direccion de casa
    list_devices = os.listdir(dir_casas[casa-1])
    max = len(list_devices)
    dir_devices = [os.path.join(dir_casas[casa-1], "channel_"+str(i) + ".dat") for i in range(1,max)]
    return dir_devices

In [ ]:
#Listas con direcciones de dispositivos por cada casa
casa_1 = directorio_dispositivos(1)
casa_2 = directorio_dispositivos(2)
casa_3 = directorio_dispositivos(3)
casa_4 = directorio_dispositivos(4)
casa_5 = directorio_dispositivos(5)
casa_6 = directorio_dispositivos(6)

In [ ]:
def imprimir_rutas(directorio):
    i = 0
    for dispositivo in directorio:
        i += 1
        print(dispositivo)
    print(f'Dispositivos totales {i} \n')
print("House_1")
imprimir_rutas(casa_1)
print("House_2\n")
imprimir_rutas(casa_2)
print("House_3\n")
imprimir_rutas(casa_3)
print("House_4\n")
imprimir_rutas(casa_4)
print("House_5\n")
imprimir_rutas(casa_5)

In [ ]:
#Extraccion de nombres de etiquetas de dipositivo por cada casa
def nombre_dispositivos(casa):
    #directorio de la casa con etiquetas
    dir_casa = dir_casas[casa-1] + "\\labels.dat"    
    #Diccionario con nombres dispositivos
    label = {}
    #Carga el directorio
    with open(dir_casa) as f:
        #Lee el archivo
        for line in f:
            #Separo las palabras mediante espacios quedando indice y nombre
            splitted_line = line.split(' ')
            #Guarda las palabras encontradas usando el indice como key y el nombre como value
            label[int(splitted_line[0])] = splitted_line[1].strip() + '_' + splitted_line[0]
    #Retorna el diccionario con las etiquetas
    return label


In [ ]:
labels_casa_1 = nombre_dispositivos(1)
labels_casa_2 = nombre_dispositivos(2)
labels_casa_3 = nombre_dispositivos(3)
labels_casa_4 = nombre_dispositivos(4)
labels_casa_5 = nombre_dispositivos(5)
labels_casa_6 = nombre_dispositivos(6)

In [ ]:
print('Casa 1\n')
print(labels_casa_1)
print(f'\nCasa 2\n')
print(labels_casa_2)
print(f'\nCasa 3\n')
print(labels_casa_3)
print(f'\nCasa 4\n')
print(labels_casa_4)
print(f'\nCasa 5\n')
print(labels_casa_5)

## Verificando datos faltantes por casa
Los valores de tiempo son consecutivos y por segundos. El objetivo ahora es verificar si hay tiempos de medida muertos, osea que los instantes en lo que se mide entre un punto y otro no son consecutivos. 
- Buscamos datos con diferencia entre medidas menores a 20 segundos. En este caso se propone rellenar los datos con el dato anterior al gap. 
- El siguiente paso consiste en remuestrear los datos cada 4 segundos, tomando la media de estos valores como dato que remplazara el original.

In [ ]:
#channel_A = pd.read_csv(casa[device],sep=r' ',encoding='utf-8',names=['UTC_Unix',label_casa[device+1]],header=None)
dispositivo_c1d1 = pd.read_csv(casa_1[0], sep=r' ', encoding='utf-8', names=['UTC_Unix',labels_casa_1[1]], header=None)

In [ ]:
dispositivo_c1d1.head()

In [ ]:
dispositivo_c1d1.isnull().sum()

In [ ]:
ts1 = 1284101485
for i in range(60):
    print(ts1)
    print(datetime.utcfromtimestamp(ts1).strftime('%Y-%m-%d %H:%M:%S'))
    ts1 = ts1 + 1

In [ ]:
tiempo = dispositivo_c1d1.iloc[:,0]

In [711]:
#Inserta el gap de valores llenos al dataframe inicial
def insert_row(idx, df, df_insert):
    dfA = df.iloc[:idx, ]
    #print(f'dfA = {dfA}')
    dfB = df.iloc[idx:, ]
    #print(f'dfB = {dfB}')
    df = dfA.append(df_insert).append(dfB).reset_index(drop = True)
    return df

In [712]:
#Función para crear los valores vacios del gap
def llenar_gaps(intervalo, valor_tiempo, valor_potencia):
    op = True
    #Primer dato que inicializa el nuevo df
    new = pd.DataFrame([[valor_tiempo,valor_potencia]],columns=['Tiempo', 'dato'])
    #Si solo necesitamos uno salimos de la funcion
    if(intervalo <= 1):
        op = False
    intervalo -= 1
    #Para crear mas de un valor
    while(op):
        valor_tiempo += 1
        new = new.append({'Tiempo': valor_tiempo, 'dato': valor_potencia}, ignore_index=True)
        intervalo -= 1
        #if intervalo == 0:
        if intervalo < 1:
            op=False
    return new

In [771]:
#Busca los gaps en el dataframe
def detectar_gaps(df_dispositivo):
    op = True
    i= 0
    tiempo = df_dispositivo.iloc[:,0]
    while (op):
        if(i==len(df_dispositivo)-1):
            #op = False
            break #Rompe de manera inmediata el ciclo
        #Comparo si los intervalos de tiempo superan gaps entre 2 y 20 segundos
        #if(tiempo[i+1] - tiempo[i]) > 1 and (tiempo[i+1] - tiempo[i]) <= 20:
        if(tiempo[i+1] - tiempo[i]) > 1 and (tiempo[i+1] - tiempo[i]) <= 6:
            t1 = tiempo[i] #Al sumar 1 indica el siguiente instante de tiempo faltante
            diferencia = tiempo[i+1] - tiempo[i] - 1 #Son la cantidad faltante de datos en el gap
            relleno = df_dispositivo.iloc[i,1] #Dato con el que se rellena los espacios faltantes
            gap_lleno = llenar_gaps(diferencia, t1+1, relleno)                                          
            df_dispositivo = insert_row(i+1, df_dispositivo, gap_lleno)
            i = i+ diferencia - 1 
            tiempo = df_dispositivo.iloc[:,0]
        i+=1
    return df_dispositivo
#detectar_gaps(dispositivo_c1d1)

In [780]:
num = pd.DataFrame({'Tiempo': [1,3,4,8,9,12,13,14,24,25,27,28,30], "dato" : [100,101,102,103,104,105,106,107,108,109,110,111,112]})
num

,Tiempo,dato
0,1,100
1,3,101
2,4,102
3,8,103
4,9,104
5,12,105
6,13,106
7,14,107
8,24,108
9,25,109


In [781]:
num = detectar_gaps(num)
num

,Tiempo,dato
0,1,100
1,2,100
2,3,101
3,4,102
4,5,102
5,6,102
6,7,102
7,8,103
8,9,104
9,10,104


## Función de suavisado
Los datos de REDD son demasiado ruidosos, por lo tanto Nascimiento propone remuestrear los datos cada 4 segundos, tomando
como valor el intervalo de la media

In [798]:
def remuestreo(df, intervalo = 4):
    inicio = 0
    fin = intervalo
    tiempo = df.iloc[:,0]

    for i in range(len(df)):
        print(f'tiempo[fin] = {tiempo[inicio+fin-1]} tiempo[inicio] = {tiempo[inicio]} diferencia = {tiempo[inicio+fin-1] - tiempo[inicio]}')
        #if(tiempo[inicio+fin-1] - tiempo[inicio]) <= 6:
        if(tiempo[i+1] - tiempo[i]) > 1 and (tiempo[i+1] - tiempo[i]) <= 6:
            print(f'Ingreso i = {i}')
            aux = df.iloc[inicio:fin, 1]
            media = aux.mean()
            print(f'media = {media}')
            df.iloc[inicio:fin, 1] = media
        inicio = fin
        fin = fin + intervalo
        print(f'inicio = {inicio} y fin = {fin} tamaño = {len(df)}')
        if fin > len(df):
            print("Break")
            break
    print(df)
        
remuestreo(num)

tiempo[fin] = 4 tiempo[inicio] = 1 diferencia = 3
Ingreso i = 0
media = 100.75
inicio = 4 y fin = 8 tamaño = 21
tiempo[fin] = 12 tiempo[inicio] = 5 diferencia = 7
inicio = 8 y fin = 12 tamaño = 21
tiempo[fin] = 29 tiempo[inicio] = 9 diferencia = 20
inicio = 12 y fin = 16 tamaño = 21


KeyError: 27

In [ ]:
from datetime import datetime
ts1 = int("1284101485")
ts2 = int("1284101486")
ts3 = int("1284101487")
# if you encounter a "year is out of range" error the timestamp
# may be in milliseconds, try `ts /= 1000` in that case
print(datetime.utcfromtimestamp(ts1).strftime('%Y-%m-%d %H:%M:%S'))
print(datetime.utcfromtimestamp(ts2).strftime('%Y-%m-%d %H:%M:%S'))
print(datetime.utcfromtimestamp(ts3).strftime('%Y-%m-%d %H:%M:%S'))